# Web Scraping Gobierno Transparente Chile
##### Exploring wage expenses in the goberment. Public information.

Beautiful soup did not work. The server time outs when you connect through bsoup.

## Trying Selenium

In [1]:
import pandas as pd
import numpy as np
import time
import unicodedata
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

import csv

import sys
sys.path.append('./lib/')
#from functions import *
from tqdm import tnrange, tqdm_notebook
from unidecode import unidecode

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import lxml.html
import lxml

In [2]:


def createCustomDataFrame():
   
    columns = ['entity', 'department', 'type_contract', 'year', 'month', 'Estamento', 'Apellido paterno', 'Apellido materno', 'Nombres', 'Grado EUS',	'Calificación profesional o formación', 'Cargo', 'Región',	'Asignaciones especiales', 'Unidad monetaria', 'Remuneración Bruta Mensualizada', 'Horas extraordinarias', 'Fecha de inicio', 'Fecha de término', 'Observaciones']
    
    df = pd.DataFrame(columns=columns)
    #df = cleanData(df)
    
    return(df)



def cleanData(df):

# Another way to do the same
#    pd.concat([df[col].astype(str).str.upper() for col in df.columns], axis=1)
    df = df.str.lower()
    df = df.str.replace('á', 'a')
    df = df.str.replace('é', 'e')
    df = df.str.replace('í', 'i')
    df = df.str.replace('ó', 'o')
    df = df.str.replace('ú', 'u')        

    df['Remuneración Bruta Mensualizada'] = df['Remuneración Bruta Mensualizada'].replace(to_replace = "\.+", value="", regex=True)

    return(df)


def getGovernmentData(output_file, url, browser, num):

    gov_data = entity_data = createCustomDataFrame()
    browser.get(url)

    url_list = []
    entities = browser.find_elements_by_class_name("primaryCat")

    counter = 0
    for e in entities:
        entity_url = e.get_attribute("href")
        url_list.append(entity_url)

    print('Entities:')
    print(url_list)
    for url in url_list[num:]: 
        getEntityData(output_file, url, browser)

def getEntityData(output_file, url, browser):
    browser.get(url)
    entity_data = createCustomDataFrame()
    
    url_list =[]
    departments = browser.find_elements_by_class_name("primaryCat")

    for d in departments:
        department_url = d.get_attribute("href")
        url_list.append(department_url)

#    print('Departments:')
#    print(url_list)
    for url in url_list:
        getDepartmentData(output_file, url, browser)

def getDepartmentData(output_file, url, browser):
    
    type_contract = ['per_planta', 'per_contrata']
    url_list = []

    for t in type_contract:
        dept_contract_link = url + "/" + t
        browser.get(dept_contract_link)
        try:
            div_years = browser.find_element_by_class_name("linksIntermedios")
            unordered_list = div_years.find_element_by_tag_name("ul")
            years_links_list = unordered_list.find_elements_by_tag_name("li")
            for l in years_links_list:
                link_anchor = l.find_element_by_tag_name("a")
                year_url = link_anchor.get_attribute("href")
                url_list.append(year_url)
        except NoSuchElementException:
            print ('No contract data in ' + url)
            g = open('./output/log_error.csv', 'a')
            g.write(url + ',' + 'No contract' + "\n");
            g.close()
        
    for url in tqdm_notebook(url_list):
        getYearData(output_file, url, browser)

        
def getYearData(output_file, url, browser):
    
    browser.get(url)
    
    monthsdata = False
    # Check if we still have to dive down into the months
    try:
        div_months = browser.find_element_by_class_name("linksIntermedios")
        monthsdata = True
    except NoSuchElementException:
        div_months = False
        #print(url + ' No months, straight to year' )
        # In this case, we go straight into the yearly tables

    url_list = []
    
    

    # If we have monthly data:
    if monthsdata:

        unordered_list = div_months.find_element_by_tag_name('ul')
        months_links_list = unordered_list.find_elements_by_tag_name('li')

        for m in months_links_list:
            month_link = m.find_element_by_tag_name('a')
            month_url = month_link.get_attribute('href')
            url_list.append(month_url)
            

    # Else fetch yearly table, we are already there
    else:
        url_list.append(url)
    
#    for url in url_list:
#        print(url)
    
    for url in url_list:  # debug purposes
        getDatainPage(output_file, url, browser)

    

def getDatainPage(output_file, url, browser):

    browser.get(url)

    table_links = []
    table_links.append(url)

    try:
        pagination = browser.find_element_by_class_name("pagination")
        pages = pagination.find_elements_by_tag_name("li")

        for page in pages:
            try:
                link_element = page.find_element_by_tag_name("a")
                link = link_element.get_attribute("href")
                table_links.append(link)
            except NoSuchElementException:
                pass

        # Remove the last element, its the back arrow
        del table_links[-1]

    except:
        print ('could not get pagination from ' + url)
        f = open('./output/log_error.csv', 'a')
        f.write(url +',' + 'Error Getting pagination' + "\n");
        f.close()


    total_tables = len(table_links)

    
    # Loop through all the pages and record data
    # If the page was already visited, carry on.
    for count, i in enumerate(table_links):
        if not (i in df_visited):
                getTableData2(output_file, i, browser)
        else:
                print('Already scraped: ' + i)

def getTableData2(output_file, url, browser):

    try:
        browser.get(url)

    except WebDriverException:
        print ('error page ' + url)
        f = open('./output/log_error.csv', 'a')
        f.write(url +',' + 'Reached Error Page' + "\n");
        f.close()
        
    #######
    ### 1 Get table metadata from the breadcrumb (Year, Department, Kind of contract, ...)
    #######

    try:
        table_location_data = browser.find_element_by_class_name("breadcrumb")
        breadcrumb_items = table_location_data.find_elements_by_tag_name("li")
        num_breadcrumbs = len(breadcrumb_items)

        entity = breadcrumb_items[1].text
        department = breadcrumb_items[2].text
        type_contract = breadcrumb_items[3].text
        year = breadcrumb_items[4].text
        if num_breadcrumbs == 6:        
            month = breadcrumb_items[5].text
        else:
            month = 'allyear'

    except (NoSuchElementException, IndexError) as err:
        print ('could not get breadcrumbs from ' + url)
        f = open('./output/log_error.csv', 'a')
        f.write(url +',' + 'Error Reading Page' + "\n");
        f.close()
        entity='entity'
        department='department'
        type_contract='type contract'
        year='year'
        month='month'

    #######
    ### 2 Get Data of table
    #######

#    try:
    root = lxml.html.fromstring(browser.page_source)
    master_list = []

    for row in root.findall('.//tbody//tr'):
        row_list = []
        cells = row.xpath('.//td')

        if len(cells)>0:
            row_list = row_list + [entity, department, type_contract, year, month] 

            for e in cells:
                text = e.xpath('text()')
                if text is None:
                    text = ''
                elif len(text) == 0:
                    text = ''
                else:
                    text = text.pop()
                row_list.append(text) 

            row_list.append(url)
            master_list.append(row_list)

#    except:
#        print(e)
#        print ('could not get table from ' + url)
#        f = open('./output/log_error.csv', 'a')
#        f.write(url +',' + 'Error Reading Table' + "\n");
#        f.close()

    #######
    ### 3 Write into csv files
    #######

    try:
        with open(output_file, "a", newline='\n') as f:
            writer = csv.writer(f)
            writer.writerows(master_list)

        a = open('./output/log_opened.csv', 'a')
        a.write(url + ',' + str( time.time()  ) + "\n");
        a.close()

    except:
        try:
            # encoding problems
            master_list = [[j.encode('ascii', 'ignore') for j in row] for row in master_list]
    
            with open(output_file, "a", newline='\n') as f:
                writer = csv.writer(f)
                writer.writerows(master_list)

            a = open('./output/log_opened.csv', 'a')
            a.write(url + ',' + str( time.time()  ) + "\n");
            a.close()

        except:
            print ('could not write data from ' + url)
            g = open('./output/log_error.csv', 'a')
            g.write(url + ',' + 'Error writing' + "\n");
            g.close()
        


    
        

# Clean data already scraped, try to re-run

In [12]:
df_scraped_1 = pd.read_csv('./output/scraped_data_20170214.csv', encoding='latin_1', header=None, index_col=0, low_memory=False)
print(df_scraped_1.shape)
df_scraped_2 = pd.read_csv('./output/scraped_data_20170212.csv', encoding='latin_1', header=None, index_col=0, low_memory=False)
print(df_scraped_2.shape)
df_scraped_3 = pd.read_csv('./output/scraped_data_20170215.csv', encoding='latin_1', header=None, index_col=0, low_memory=False)
print(df_scraped_3.shape)
df_scraped_4 = pd.read_csv('./output/scraped_data_20170215-1.csv', encoding='latin_1', header=None, index_col=0, low_memory=False)
print(df_scraped_4.shape)
df_scraped_5 = pd.read_csv('./output/scraped_data_20170216.csv', encoding='latin_1', header=None, index_col=0, low_memory=False)
print(df_scraped_5.shape)
df_scraped_6 = pd.read_csv('./output/scraped_data_20170221.csv', encoding='latin_1', header=None, index_col=0, low_memory=False)
print(df_scraped_6.shape)

(44783, 21)
(25227, 21)
(50586, 20)
(3145, 20)
(36591, 20)
(15300, 20)


In [13]:
df_scraped = pd.concat([df_scraped_1, df_scraped_2, df_scraped_3, df_scraped_4, df_scraped_5, df_scraped_6]).drop_duplicates()
df_scraped.shape

(114551, 21)

In [34]:
df_visited = []

In [3]:
df_visited_1 = pd.read_csv('./output/log_opened_20170212.csv', header=None).ix[:,0].drop_duplicates().tolist()
df_visited_2 = pd.read_csv('./output/log_opened_20170214.csv', header=None).ix[:,0].drop_duplicates().tolist()
df_visited_3 = pd.read_csv('./output/log_opened_20170215.csv', header=None).ix[:,0].drop_duplicates().tolist()
df_visited_4 = pd.read_csv('./output/log_opened_20170221.csv', header=None).ix[:,0].drop_duplicates().tolist()
df_visited_5 = pd.read_csv('./output/log_opened_20170224.csv', header=None).ix[:,0].drop_duplicates().tolist()
df_visited_6 = pd.read_csv('./output/log_opened_20170225.csv', header=None).ix[:,0].drop_duplicates().tolist()


df_visited = list(set(df_visited_1 + df_visited_2 + df_visited_3 + df_visited_4 + df_visited_5 + df_visited_6))
print(len(df_visited))
print(len(df_visited_1))
print(len(df_visited_2))
print(len(df_visited_3))
print(len(df_visited_4))
print(len(df_visited_5))
print(len(df_visited_6))




14746
168
482
1797
156
1031
11130


In [4]:
output = './output/scraped_data.csv'

url = 'http://www.gobiernotransparentechile.cl/'
browser = webdriver.Firefox()

df = getGovernmentData(output, url, browser, 14)

Entities:
['http://www.gobiernotransparentechile.cl/directorio/entidad/1', 'http://www.gobiernotransparentechile.cl/directorio/entidad/2', 'http://www.gobiernotransparentechile.cl/directorio/entidad/3', 'http://www.gobiernotransparentechile.cl/directorio/entidad/4', 'http://www.gobiernotransparentechile.cl/directorio/entidad/5', 'http://www.gobiernotransparentechile.cl/directorio/entidad/6', 'http://www.gobiernotransparentechile.cl/directorio/entidad/7', 'http://www.gobiernotransparentechile.cl/directorio/entidad/8', 'http://www.gobiernotransparentechile.cl/directorio/entidad/9', 'http://www.gobiernotransparentechile.cl/directorio/entidad/10', 'http://www.gobiernotransparentechile.cl/directorio/entidad/11', 'http://www.gobiernotransparentechile.cl/directorio/entidad/12', 'http://www.gobiernotransparentechile.cl/directorio/entidad/13', 'http://www.gobiernotransparentechile.cl/directorio/entidad/14', 'http://www.gobiernotransparentechile.cl/directorio/entidad/15', 'http://www.gobiernotra

IndexError: list index out of range

# Debugging


In [94]:
browser_debug = webdriver.Firefox()
dfDebug = getTableData('./output/debugging.csv', 'http://www.gobiernotransparentechile.cl/directorio/entidad/2/2/per_contrata/2009/Abril', browser_debug)

6
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="b4a9c7a6-da33-47af-a779-5a5b23f32ee6", element="75c2269d-c5d3-47a7-b0ae-2718a977c811")>
Text: Abril


In [138]:
browser_debug = webdriver.Firefox()
dfDebug = getDepartmentData('./output/debugging_dept.csv', 'http://www.gobiernotransparentechile.cl/directorio/entidad/2/4/', browser_debug)

No contract data in http://www.gobiernotransparentechile.cl/directorio/entidad/2/4/
No contract data in http://www.gobiernotransparentechile.cl/directorio/entidad/2/4/


In [93]:
dfDebug

,entity,department,type_contract,year,month,Estamento,Apellido paterno,Apellido materno,Nombres,Grado EUS,...,Cargo,Región,Asignaciones especiales,Unidad monetaria,Remuneración Bruta Mensualizada,Horas extraordinarias,Fecha de inicio,Fecha de término,Observaciones,url
0,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,CARRASCO,SEGURA,WALDO FRANCISCO JAVIER,3,...,JEFE GABINETE SUBSECRETARIO DEL INTERIOR,,,,0,,01/01/2009,31/12/2009,PERCIBE ASIGNACIN POR FUNCIN CRITICA 100%,http://www.gobiernotransparentechile.cl/direct...
1,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,JOIGNANT,MUOZ,MARTA IVETTE,3,...,JEFE PROGRAMA EXONERADOS,,,,0,,01/01/2009,31/12/2009,PERCIBE ASIGNACIN POR FUNCIN CRITICA 55%,http://www.gobiernotransparentechile.cl/direct...
2,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,LAGOS,SCHUFFENEGER,HUMBERTO,3,...,"EXPERTO EN SECTAS, OFICINA DE ASUNTOS RELIGIOS...",,,,0,,01/01/2009,31/12/2009,2,http://www.gobiernotransparentechile.cl/direct...
3,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,ROJAS,FLORES,JAIME DAVID,3,...,ASESOR SISTEMA INTEGRADO DE EMERGENCIA,,,,0,,01/01/2009,31/12/2009,PERCIBE ASIGNACIN POR FUNCIN CRITICA 90%,http://www.gobiernotransparentechile.cl/direct...
4,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,BRAVO,GOLDSMITH,CARLOS ALBERTO,4,...,JEFE UNIDAD ELECTORAL,,,,0,,01/01/2009,31/12/2009,PERCIBE ASIGNACIN POR FUNCIN CRITICA 18%,http://www.gobiernotransparentechile.cl/direct...
5,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,PROFESIONAL,GONZALEZ,LEVER,PAZ MATILDE,4,...,ASESOR SECCION ESTUDIOS DEPARTAMENTO DE EXTRAN...,,,,0,,01/01/2009,31/12/2009,1,http://www.gobiernotransparentechile.cl/direct...
6,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,PROFESIONAL,INOSTROZA,PAREDES,CLAUDIA CAROLINA,4,...,AUDITOR MINISTERIAL,,,,0,,01/01/2009,31/12/2009,,http://www.gobiernotransparentechile.cl/direct...
7,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,DANTON,QUIROZ,MONICA ELIANA,5,...,ASISTENTE MINISTRO,,,,0,,01/01/2009,31/12/2009,1,http://www.gobiernotransparentechile.cl/direct...
8,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,HERNANDEZ,CISTERNAS,CLAUDIA TAMARA,5,...,ASISTENTE SOCIAL DIVISION DE ADMINISTRACION Y ...,,,,0,,01/01/2009,31/12/2009,,http://www.gobiernotransparentechile.cl/direct...
9,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,PROFESIONAL,ATENAS,SALDIA,FRANCISCO ARNOLDO,5,...,"JEFE DEPARTAMENTO PLANIFICACION, ESTUDIOS Y CO...",,,,0,,01/01/2009,31/12/2009,1,http://www.gobiernotransparentechile.cl/direct...


In [95]:
dfDebug1 = dfDebug[:]


for col in dfDebug1.columns[0:21]:
    dfDebug1.ix[:,col] = dfDebug1.ix[:,col].str.encode('ascii', 'ignore')
    dfDebug1.ix[:,col] = dfDebug1.ix[:,col].str.decode('ascii', 'ignore')
        

In [96]:
dfDebug1

,entity,department,type_contract,year,month,Estamento,Apellido paterno,Apellido materno,Nombres,Grado EUS,...,Cargo,Región,Asignaciones especiales,Unidad monetaria,Remuneración Bruta Mensualizada,Horas extraordinarias,Fecha de inicio,Fecha de término,Observaciones,url
0,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,CARRASCO,SEGURA,WALDO FRANCISCO JAVIER,3,...,JEFE GABINETE SUBSECRETARIO DEL INTERIOR,,,,0,,01/01/2009,31/12/2009,PERCIBE ASIGNACIN POR FUNCIN CRITICA 100%,http://www.gobiernotransparentechile.cl/direct...
1,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,JOIGNANT,MUOZ,MARTA IVETTE,3,...,JEFE PROGRAMA EXONERADOS,,,,0,,01/01/2009,31/12/2009,PERCIBE ASIGNACIN POR FUNCIN CRITICA 55%,http://www.gobiernotransparentechile.cl/direct...
2,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,LAGOS,SCHUFFENEGER,HUMBERTO,3,...,"EXPERTO EN SECTAS, OFICINA DE ASUNTOS RELIGIOS...",,,,0,,01/01/2009,31/12/2009,2,http://www.gobiernotransparentechile.cl/direct...
3,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,ROJAS,FLORES,JAIME DAVID,3,...,ASESOR SISTEMA INTEGRADO DE EMERGENCIA,,,,0,,01/01/2009,31/12/2009,PERCIBE ASIGNACIN POR FUNCIN CRITICA 90%,http://www.gobiernotransparentechile.cl/direct...
4,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,BRAVO,GOLDSMITH,CARLOS ALBERTO,4,...,JEFE UNIDAD ELECTORAL,,,,0,,01/01/2009,31/12/2009,PERCIBE ASIGNACIN POR FUNCIN CRITICA 18%,http://www.gobiernotransparentechile.cl/direct...
5,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,PROFESIONAL,GONZALEZ,LEVER,PAZ MATILDE,4,...,ASESOR SECCION ESTUDIOS DEPARTAMENTO DE EXTRAN...,,,,0,,01/01/2009,31/12/2009,1,http://www.gobiernotransparentechile.cl/direct...
6,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,PROFESIONAL,INOSTROZA,PAREDES,CLAUDIA CAROLINA,4,...,AUDITOR MINISTERIAL,,,,0,,01/01/2009,31/12/2009,,http://www.gobiernotransparentechile.cl/direct...
7,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,DANTON,QUIROZ,MONICA ELIANA,5,...,ASISTENTE MINISTRO,,,,0,,01/01/2009,31/12/2009,1,http://www.gobiernotransparentechile.cl/direct...
8,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,EXPERTO,HERNANDEZ,CISTERNAS,CLAUDIA TAMARA,5,...,ASISTENTE SOCIAL DIVISION DE ADMINISTRACION Y ...,,,,0,,01/01/2009,31/12/2009,,http://www.gobiernotransparentechile.cl/direct...
9,Ministerio del Interior y Seguridad Pblica,Subsecretara del Interior,Dotacin a contrata,2009,Abril,PROFESIONAL,ATENAS,SALDIA,FRANCISCO ARNOLDO,5,...,"JEFE DEPARTAMENTO PLANIFICACION, ESTUDIOS Y CO...",,,,0,,01/01/2009,31/12/2009,1,http://www.gobiernotransparentechile.cl/direct...


In [52]:
dfDebug.ix[0,'month']

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="8503552e-6cc2-4af8-985c-4ae80f7e3fb3", element="9111fc7f-2676-421b-a2c5-f13ed5e58028")>

In [38]:
dfDebug.to_csv('./output/debugTest.csv')

In [9]:
a = 'xx'
print(a)
print(a.encode('ascii', 'ignore'))

xx
b'xx'


In [18]:
a = [['a', 'b'], ['c', 'd']]
b = pd.DataFrame(a, columns=['uno', 'dos'])
#for x in b.columns:
#    b[x] = b[x].str.decode('ascii', 'ignore')



In [25]:
b.uno.str.encode('ascii')

0    b'a'
1    b'c'
Name: uno, dtype: object